# dge-plots.ipynb
Generate upsetplot, heatmap for the DGE analysis in deseq2_shL1CAM-shSHTN1.ipynb

## Requirements
- Package dependencies in `heatmap.yml`
- DGE analysis output files in `results/deseq2/sh{GENE}_deg.tsv`

In [ ]:
# Load dependencies
Sys.setenv(LANGUAGE = "en") # set language to "ja" if you prefer
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(grid))
suppressPackageStartupMessages(library(ComplexHeatmap))
suppressPackageStartupMessages(library(extrafont))
suppressPackageStartupMessages(library(svglite))

getwd()
sessionInfo()

In [ ]:
# Specify global plotting settings
suppressMessages(extrafont::font_import(pattern="Arial",prompt=FALSE))
suppressMessages(extrafont::loadfonts())
gpmaj = grid::gpar(fontsize = 14, fontfamily = "Arial")
gpmin = grid::gpar(fontsize = 10, fontfamily = "Arial")

save_heatmap <- function(hm, filename_base, width = unit(5,"cm"), height = unit(5,"cm"), res = 300) {
  # Save as PNG
  png(paste0(filename_base, ".png"), width = width, height = height, units = "in", res = res)
  draw(hm)
  dev.off()
  
  # Save as SVG
  svg(paste0(filename_base, ".svg"), width = width, height = height)
  draw(hm)
  dev.off()
}

In [ ]:
# Load data
shtn1_file = 'results/deseq2/shSHTN1_batch4_deg.tsv'
l1cam_file = 'results/deseq2/shL1CAM_batch4_deg.tsv'

read_dge <- function(file){
    data = read.table(file, header = TRUE, sep = "\t", row.names = 1) %>%
        rownames_to_column('gene') %>%
        as_tibble
    return(data)
}
filter_dge <- function(data,direction='down',alpha=0.05){
    if (direction == 'down'){
        data = data %>%
            filter(log2FoldChange < 0, padj < alpha) %>%
            arrange(log2FoldChange)
        return(data)
    } else if (direction == 'up'){
        data = data %>%
            filter(log2FoldChange > 0, padj < alpha) %>%
            arrange(desc(log2FoldChange))
        return(data)
    } else {
        stop("direction must be 'up' or 'down'")
    }
}

shtn1_data = read_dge(shtn1_file)
l1cam_data = read_dge(l1cam_file)

# get all genes tested
universe=shtn1_data$gene %>% unique
stopifnot(setequal(universe, l1cam_data$gene %>% unique))

shtn1_hits_dn = shtn1_data %>% filter_dge
l1cam_hits_dn = l1cam_data %>% filter_dge

shtn1_hits_up = shtn1_data %>% filter_dge(direction='up')
l1cam_hits_up = l1cam_data %>% filter_dge(direction='up')

shtn1_data %>% head(n=10)
l1cam_data %>% head(n=10)

In [ ]:
upset_data = make_comb_mat(list(
    "shSHTN1\ndown" = shtn1_hits_dn$gene %>% unique,
    "shL1CAM\ndown" = l1cam_hits_dn$gene %>% unique
),universal_set=universe,mode='distinct')


fig1 <- UpSet(upset_data,
    width=unit(5,"cm"),
    height=unit(5,"cm"),
    top_annotation = upset_top_annotation(upset_data, 
        add_numbers = TRUE, 
        height=unit(6,"cm"),
        annotation_name_gp=gpmaj,
        annotation_name_rot=90,
        numbers_gp=gpmin,
        axis_param=list(gp=gpmin)
    ),
    right_annotation = upset_right_annotation(upset_data, 
        add_numbers = TRUE,
        width=unit(6,"cm"),
        annotation_name_gp=gpmaj,
        numbers_gp=gpmin,
        axis_param=list(gp=gpmin)
    ),
    row_names_gp = gpmaj,
    column_names_gp = gpmaj
)
fig1
save_heatmap(hm = fig1, filename_base = 'results/deseq2/upset_down')

In [ ]:
fig1 %>% class

In [ ]:
upset_data = make_comb_mat(list(
    "shSHTN1\ndown" = shtn1_hits_dn$gene %>% unique,
    "shL1CAM\ndown" = l1cam_hits_dn$gene %>% unique,
    "shSHTN1\nup" = shtn1_hits_up$gene %>% unique,
    "shL1CAM\nup" = l1cam_hits_up$gene %>% unique
),universal_set=universe,mode='distinct')


fig2 <- UpSet(upset_data,
    width=unit(5,"cm"),
    height=unit(5,"cm"),
    top_annotation = upset_top_annotation(upset_data, 
        add_numbers = TRUE, 
        height=unit(6,"cm"),
        annotation_name_gp=gpmaj,
        annotation_name_rot=90,
        numbers_gp=gpmin,
        axis_param=list(gp=gpmin)
    ),
    right_annotation = upset_right_annotation(upset_data, 
        add_numbers = TRUE,
        width=unit(6,"cm"),
        annotation_name_gp=gpmaj,
        numbers_gp=gpmin,
        axis_param=list(gp=gpmin)
    ),
    row_names_gp = gpmaj,
    column_names_gp = gpmaj
)
fig2
save_heatmap(hm = fig2, filename_base = 'results/deseq2/upset_both')

In [ ]:
hits <- intersect(shtn1_hits_dn$gene, l1cam_hits_dn$gene) 
writeLines(as.character(hits))

In [ ]:
writeLines(as.character(shtn1_hits_dn$gene))

In [ ]:
# Lookup genes of interest
genes_of_interest = c('SHTN1','L1CAM','CCN2','CCN1','YAP1') # CTGF = CCN2; CYR61 =  CCN1
shtn1_data %>% subset(gene %in% genes_of_interest)
message("")
l1cam_data %>% subset(gene %in% genes_of_interest)